In [3]:
import openai

# Initialize the OpenAI client
client = openai.OpenAI()

# Define the prompt as a text string
prompt = """
Given a hypnokink headspace, generate a detailed ontology including the headspace description, psychological appeal explanation, a concise set of thematic keywords, and functional metadata.

Input:
- Kink Name: {kink_name}
- Guidance: {guidance}

Output (in JSON format):
{{
    "description": "A vivid and detailed description defining the desired hypnotic headspace. Focus on the subjective experience, emotional tone, and how the subject feels or behaves while immersed in the headspace. Describe how this state unfolds or is induced, and what it might feel like to someone going through it.",
    
    "appeal": "An insightful explanation describing the core psychological or emotional reasons individuals might find this hypnokink fulfilling or appealing. Emphasize what makes the state attractive or desirable, especially how it may serve as an emotional outlet or contrast to day-to-day responsibilities. Avoid assuming specific narratives unless implied by the guidance.",
    
    "keywords": ["A concise, curated list of approximately 10–15 simple, universally understood keywords or short phrases that reflect the emotional tone, behaviors, and symbolic language of the kink. These should be suitable for embedding as hypnotic triggers, mantra repetitions, or thematic reinforcement within scripts. Avoid overly complex or esoteric terms."],
    
    "tags": ["List one or more names from the approved tag list below, based on what categories best apply to this kink."],

    "cnc": false  // Set to true only if the theme includes elements of implied non-consent, overwhelming compulsion, helplessness, or psychological manipulation that may require a consent-education warning. Do not set true for general submission, obedience, or control unless there is a loss of agency or unsafe framing.
}}

Tag Options:
- Behavior: Themes involving actions, habits, or routines. Often includes physical or observable behaviors that can be guided or reinforced in trance.
- D/s: Themes of power exchange, obedience, control, and dominance or submission dynamics. Includes emotional surrender, rules, or training.
- Experience: Themes about the subjective state or emotional atmosphere. Often includes sensations, mood, headspace, and immersion.
- Hypnosis: Themes involving the mechanics or structure of trance. Includes things like suggestion, memory play, focus, confusion, and altered mental states.
- Identity: Themes about roles, archetypes, or transformations in self-concept. Includes persona adoption and roleplay (e.g., doll, pet, cheerleader).
- Personality: Themes about emotions, traits, or mindset shaping. Often uses affirmations or guidance to reinforce a way of feeling or being.
- Meta: Themes that are about trance itself or hypnotic process manipulation — including loops, mantras, suggestion training, or trance layering.
"""


def generate_hypnokink_ontology(kink_name, guidance, max_tokens=1000):
    formatted_prompt = prompt.format(kink_name=kink_name, guidance=guidance)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant skilled in creating hypnokink ontologies."},
            {"role": "user", "content": formatted_prompt}
        ],
        response_format={"type": "json_object"},
        max_tokens=max_tokens,
        temperature=0 # we're doing ontologies here, so we don't want any randomness
    )
    return response.choices[0].message.content


In [5]:
kink_name = "Mindlessness"
guidance = "The subject feels empty, blank, and devoid of thoughts or self-awareness"
ontology = generate_hypnokink_ontology(kink_name, guidance)
print(ontology)

{
    "description": "In the Mindlessness headspace, the subject experiences a profound sense of emptiness and tranquility, as if their mind has been gently wiped clean of all thoughts and self-awareness. This state is often induced through a series of calming suggestions and repetitive mantras that guide the subject into a deep trance. As they sink deeper, they feel their thoughts slowly dissipate, leaving behind a serene void. The subject may feel as though they are floating in a vast, quiet space, free from the burdens of decision-making or self-reflection. This headspace is characterized by a sense of peaceful detachment, where the subject is fully present in the moment yet blissfully unaware of their surroundings or identity. The experience is akin to being in a dreamlike state, where the mind is at rest and the body is relaxed, allowing the subject to simply exist without the need for conscious thought or action.",
    
    "appeal": "The appeal of the Mindlessness headspace lies

In [ ]:
kink_name = "Amnesia"
guidance = "Forgetting past thoughts, memories, and experiences, creating a blank slate."
ontology = generate_hypnokink_ontology(kink_name, guidance)
print(ontology)

In [ ]:
import os
import json

def crawl_for_txt_files(root_path):
    kink_guidance_pairs = []
    for dirpath, _, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.endswith(".txt"):
                kink_name = os.path.splitext(filename)[0]
                guidance = None
                file_path = os.path.join(dirpath, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    for line in file:
                        if line.startswith("Description:"):
                            guidance = line[len("Description:"):].strip()
                            break
                if guidance:
                    kink_guidance_pairs.append((kink_name, guidance))
    return kink_guidance_pairs

# Change 'YOUR_PATH_HERE' to your specific start directory
directory_path = "C:\\Users\\conno\\Code\\ai-conditioner-web\\hypnosis\\mantras\\"
pairs = crawl_for_txt_files(directory_path)

kink_guidance_list = []
for kink_name, guidance in pairs:
    kink_guidance_list.append({"kink_name": kink_name, "guidance": guidance})

# Print the list to verify
# Pretty print the kink_guidance_list
print(json.dumps(kink_guidance_list, indent=4, ensure_ascii=False))

In [ ]:
# Create output folder for the JSON files
output_folder = "ontologies"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Define the keys that each ontology json should have
required_keys = ["description", "appeal", "keywords", "tags", "cnc"]

# Loop through each kink in the list
for item in kink_guidance_list:
    kink_name = item["kink_name"]
    guidance = item["guidance"]
    try:
        # Generate the ontology using the pre-defined function
        ontology_str = generate_hypnokink_ontology(kink_name, guidance)
        ontology_json = json.loads(ontology_str)
        
        # Check if all required keys are present
        if not all(key in ontology_json for key in required_keys):
            print(f"Ontology for '{kink_name}' is missing some required keys. Skipping.")
            continue
        
        # Save the parsed JSON to a file named [kink_name].json in the output folder
        file_path = os.path.join(output_folder, f"{kink_name}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(ontology_json, f, indent=4, ensure_ascii=False)
        print(f"Ontology for '{kink_name}' saved to {file_path}.")
    except Exception as e:
        print(f"Error processing '{kink_name}': {e}")

In [ ]:
# Create a dictionary to map each meta category (i.e. tag) to a list of kink names.
meta_categories = {}

# Iterate over all JSON files in the output folder
for filename in os.listdir(output_folder):
    if filename.endswith(".json"):
        kink_name = os.path.splitext(filename)[0]
        file_full_path = os.path.join(output_folder, filename)
        with open(file_full_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # Get the tags for this kink
            for tag in data.get("tags", []):
                if tag not in meta_categories:
                    meta_categories[tag] = []
                meta_categories[tag].append(kink_name)

# Pretty print the result
print(json.dumps(meta_categories, indent=4))